In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Download punkt data from nltk library. This data is needed for the tokeninizer of the character.
# Download stopwords using the nltk library.

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
df_train = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/train.csv",encoding="latin-1")
df_test = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/test.csv",encoding="latin-1")
df_train.Data

In [ ]:
X_train = df_train["Data"]
y_train = df_train["Class"]
X_test = df_test["Data"]

# Text data preprocessing

In [ ]:
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
def data_processing(text):
    # If the input text is not an English character, it is changed to ' '.
    pre_words = re.sub(pattern = '[^A-Za-z]', repl = ' ', string = text)
    
    # Convert uppercase letters to lowercase letters
    pre_words = pre_words.lower()
    
    # Tokenize pre_word to list words and store them in tokenized_words
    tokenized_words = word_tokenize(pre_words)
    
    # Find stopwords in English using the words() function of stopwords and store them in stops.
    stops = nltk.corpus.stopwords.words('english')
    
    tokenized_words_remove = []
    
    for w in tokenized_words:
        # If there is no stopword in tokenized_words w of tokenized word list, append to tokenized_words_remove list
        if w not in stops:
            tokenized_words_remove.append(w)
    
    stemmer = PorterStemmer()
    for i in range(len(tokenized_words_remove)):
        # Using the stem built-in function in PorterStemmer
        # it goes through the process of changing the word with the same meaning to the same word in the tokenized_words_remove list.
        tokenized_words_remove[i] = stemmer.stem(tokenized_words_remove[i])
    
    return ( " ".join( tokenized_words_remove ))

In [ ]:
X_train=[data_processing(i) for i in X_train]
X_test=[data_processing(i) for i in X_test]

# Bag of Word 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 100) # Define CountVectorizer or TfidfVectorizer
# The parameter 'max_features = 100' means to extract as many as 100 corpus in the order of the highest frequency.

X_train = np.array(X_train, dtype = 'U')
X_test = np.array(X_test, dtype = 'U')

X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [ ]:
y_train[y_train=="ham"] = 0
y_train[y_train=="spam"] = 1
y_train = y_train.astype("uint8")

# SVM: classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC(gamma = 'auto', C = 100) # Setting the parameters

# Parameter setting history
# gamma = 'auto' // kaggle : 0.94439
# gamma = 'auto', C = 5 // kaggle : 0.95695
# gamma = 'auto', C = 10 // kaggle : 0.95784
# gamma = 'auto', C = 100 // kaggle : 0.96322 **
# gamma = 'auto', C = 1000 // kaggle : 0.96143
# gamma = 'auto', C = 100, kernel = 'linear'// kaggle: 0.95874
# MaxAbsScaler(), gamma = 'auto', C = 100 // kaggle: 0.95695
# gamma = 'auto', C = 100, kernel = 'sigmoid' // kaggle: 0.96053
# gamma = 'auto', C = 10, kernel = 'sigmoid' // kaggle: 0.95784
# gamma = 'auto'//0.96233
# gamma = 'auto', class_weight = 'balanced' //0.95605


svc.fit(X_train_features, y_train)
y_pred = svc.predict(X_test_features)

# Predict to CSV

In [ ]:
df_pred={"ID": range(np.array(y_pred).shape[0]),"Class":y_pred}
df_pred=pd.DataFrame(df_pred)
df_pred.to_csv("predict.csv",index=False)